In [51]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = '3ac843f881d943ff8cb08a2f660fc58e'
client_secret = 'e409995d02ba4c038fcd22b4b10c4ca6'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

import pandas as pd 
import time 

In [52]:
import requests
import datetime
from urllib.parse import urlencode
import base64

In [53]:
class SpotifyAPI(object):    
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [54]:
spotify = SpotifyAPI(client_id, client_secret)

In [55]:
import pandas as pd
df_bands = pd.read_csv('cincy_music_band_list.csv')
df_bands.columns = ['order', 'names', 'genres', 'favorites']
df_bands.head()

,order,names,genres,favorites
0,0,Trauma Illinois,rock\nalternative,1.0
1,1,Ashley Lilly,acoustic\nsinger/songwriter\nindie\nalternativ...,NaN
2,2,Fam Band,rock\npop\nhip hop,69.0
3,3,Saving Escape,rock\ngarage rock,2.0
4,4,The Tillers,acoustic\nfolk\nbluegrass,NaN


In [56]:
# df_add_bands = pd.ExcelFile("add_cincy_bands.xlsx")\

# df_add_bands=pd.read_excel(
#      os.path.join("add_cincy_bands.xlsx"),
#      engine='openpyxl',
# )

df_add_bands = pd.read_csv('add_cincy_bands.csv')

df_add_bands.head(5)

,BANDS
0,The Grove
1,The Midwestern
2,Sundae Drives
3,500 Miles to Memphis
4,See You In The Funnies


In [57]:
df_scraped_ids = pd.read_csv('ids_scraped.csv')
df_scraped_ids.head(5)

,Unnamed: 0,spotify,id
0,0,https://open.spotify.com/artist/7GT8mZHoTsRGMk...,GT8mZHoTsRGMkmownzCeo
1,1,https://open.spotify.com/artist/2VfhnjHV7e3M49...,VfhnjHV7e3M499VPa1w71?
2,2,https://open.spotify.com/artist/0DJJrg8eUagEWZ...,DJJrg8eUagEWZXFgwKJfM
3,3,https://open.spotify.com/artist/3KQoZQIHTfe9Fs...,KQoZQIHTfe9FsMMRuw9S6?
4,4,https://open.spotify.com/artist/4bnCmwKU89Rt1z...,bnCmwKU89Rt1zWnRIdSUa?


In [58]:
search = spotify.search(query="Trauma Illinois", search_type="artist")

q=Trauma+Illinois&type=artist


In [59]:
spotify.search({"artist": "7GT8mZHoTsRGMkmownzCeo"}, search_type="artist")

q=artist%3A7GT8mZHoTsRGMkmownzCeo&type=artist


{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3A7GT8mZHoTsRGMkmownzCeo&type=artist&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [60]:
# search = spotify.search(query="7GT8mZHoTsRGMkmownzCeo", search_type="id")
# # get_artist('7GT8mZHoTsRGMkmownzCeo')
search = spotify.search(query="7GT8mZHoTsRGMkmownzCeo", search_type="artist")

q=7GT8mZHoTsRGMkmownzCeo&type=artist


In [61]:
# id = '7GT8mZHoTsRGMkmownzCeo'
# https://api.spotify.com/v1/artists/{id}/albums

In [62]:
pd.DataFrame(search.items())

,0,1
0,artists,{'href': 'https://api.spotify.com/v1/search?qu...


In [63]:
search

{'artists': {'href': 'https://api.spotify.com/v1/search?query=7GT8mZHoTsRGMkmownzCeo&type=artist&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [64]:
df_bands = df_bands[['names']]
df_bands.head()

,names
0,Trauma Illinois
1,Ashley Lilly
2,Fam Band
3,Saving Escape
4,The Tillers


In [65]:
df_add_bands = df_add_bands.rename(columns={'BANDS': 'names'})
df_merge_bands = pd.concat([df_bands,df_add_bands], axis=0)
df_merge_bands.head(5)

,names
0,Trauma Illinois
1,Ashley Lilly
2,Fam Band
3,Saving Escape
4,The Tillers


In [66]:
len(df_merge_bands)

757

In [67]:
len(df_bands)

658

In [68]:
# spotify.search(query="Fam Band", search_type="artist")

df_spotify_info = pd.DataFrame(columns=["artist"])

# for row in df_bands.names():
#     name = row
#     spotify.search(query=name, search_type="artist")

for index, row in df_merge_bands.iterrows():
    print(row['names'])
    append = spotify.search(query=(row['names']), search_type="artist")
    df_spotify_info = df_spotify_info.append({"artist":append}, ignore_index=True)

# for band in band_test:
#     print(band.text)
#     df_likes = df_spotify.append({"spotify":band.text}, ignore_index=True)

usion
q=Johnny+Fink+and+The+Intrusion&type=artist
Clark Jenkins
q=Clark+Jenkins&type=artist
Jonathan Cody White
q=Jonathan+Cody+White&type=artist
Cousin Kissers
q=Cousin+Kissers&type=artist
Two Inch Winky
q=Two+Inch+Winky&type=artist
Honey Hush
q=Honey+Hush&type=artist
Brother Airborne
q=Brother+Airborne&type=artist
Veronica Grim
q=Veronica+Grim&type=artist
Moonshine Drive
q=Moonshine+Drive&type=artist
Queen City Cabaret
q=Queen+City+Cabaret&type=artist
Sick Serenity
q=Sick+Serenity&type=artist
DIAMOND JIM DEWS
q=DIAMOND+JIM+DEWS&type=artist
Cheryl Renee
q=Cheryl+Renee&type=artist
Continua
q=Continua&type=artist
The Dunby Four
q=The+Dunby+Four&type=artist
Xero Hour
q=Xero+Hour&type=artist
Sarah Gorak
q=Sarah+Gorak&type=artist
The American Scheme
q=The+American+Scheme&type=artist
Flesh Mother
q=Flesh+Mother&type=artist
Dark Matters
q=Dark+Matters&type=artist
Ricky and the Wild
q=Ricky+and+the+Wild&type=artist
Luscious Dogs
q=Luscious+Dogs&type=artist
Quotah
q=Quotah&type=artist
A Plea F

In [69]:
pd.set_option('max_colwidth', -1)
df_spotify_info.head(1)

,artist
0,"{'artists': {'href': 'https://api.spotify.com/v1/search?query=Trauma+Illinois&type=artist&offset=0&limit=20', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'}, 'followers': {'href': None, 'total': 520}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo', 'id': '7GT8mZHoTsRGMkmownzCeo', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/6990911ce4283e2c2025ad9889542a9a98272789', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/b43c21f903c8e81dac4c84f0d74b8ece139cf759', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ec04bda28f9d657d3358fc6472b6e4b43058597a', 'width': 160}], 'name': 'Trauma Illinois', 'popularity': 4, 'type': 'artist', 'uri': 'spotify:artist:7GT8mZHoTsRGMkmownzCeo'}], 'limit': 20, 'next': None, 'offset': 0, 'previous': None, 'total': 1}}"


In [70]:
# df_spotify_info['artist'].apply(pd.Series)
test = pd.json_normalize(df_spotify_info['artist'])
# pd.json_normalize(test['artists.items'])
test = test['artists.items'].apply(pd.Series)
test.columns = ['Main', 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
test.columns
test
trim_spotify = test['Main'].apply(pd.Series)
trim_spotify.head(2)

,0,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,NaN,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},"{'href': None, 'total': 520}",[],https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,"[{'height': 640, 'url': 'https://i.scdn.co/image/6990911ce4283e2c2025ad9889542a9a98272789', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/b43c21f903c8e81dac4c84f0d74b8ece139cf759', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ec04bda28f9d657d3358fc6472b6e4b43058597a', 'width': 160}]",Trauma Illinois,4.0,artist,spotify:artist:7GT8mZHoTsRGMkmownzCeo
1,NaN,{'spotify': 'https://open.spotify.com/artist/4Fo2qgLwTad6ICYbIu7CQR'},"{'href': None, 'total': 747}",[movie tunes],https://api.spotify.com/v1/artists/4Fo2qgLwTad6ICYbIu7CQR,4Fo2qgLwTad6ICYbIu7CQR,[],Ashley Lilley,57.0,artist,spotify:artist:4Fo2qgLwTad6ICYbIu7CQR


In [71]:
trim_spotify['followers'] = trim_spotify['followers'].astype(str)
trim_spotify['popularity'] = trim_spotify['popularity'].astype(str)
trim_spotify.dtypes

0                float64
external_urls    object 
followers        object 
genres           object 
href             object 
id               object 
images           object 
name             object 
popularity       object 
type             object 
uri              object 
dtype: object

In [72]:
## Trim Dictionary to just values
trim_spotify['followers'] = trim_spotify['followers'].str[-4:]
trim_spotify['followers'] = trim_spotify['followers'].str[:3]
trim_spotify.head(2)

,0,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,NaN,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},520,[],https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,"[{'height': 640, 'url': 'https://i.scdn.co/image/6990911ce4283e2c2025ad9889542a9a98272789', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/b43c21f903c8e81dac4c84f0d74b8ece139cf759', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ec04bda28f9d657d3358fc6472b6e4b43058597a', 'width': 160}]",Trauma Illinois,4.0,artist,spotify:artist:7GT8mZHoTsRGMkmownzCeo
1,NaN,{'spotify': 'https://open.spotify.com/artist/4Fo2qgLwTad6ICYbIu7CQR'},747,[movie tunes],https://api.spotify.com/v1/artists/4Fo2qgLwTad6ICYbIu7CQR,4Fo2qgLwTad6ICYbIu7CQR,[],Ashley Lilley,57.0,artist,spotify:artist:4Fo2qgLwTad6ICYbIu7CQR


In [73]:
## Drop unnecessary columns
trim_spotify_clean = trim_spotify.drop([0, 'images','type','uri'], axis=1)
trim_spotify_clean.head(5)

,external_urls,followers,genres,href,id,name,popularity
0,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},520,[],https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,Trauma Illinois,4.0
1,{'spotify': 'https://open.spotify.com/artist/4Fo2qgLwTad6ICYbIu7CQR'},747,[movie tunes],https://api.spotify.com/v1/artists/4Fo2qgLwTad6ICYbIu7CQR,4Fo2qgLwTad6ICYbIu7CQR,Ashley Lilley,57.0
2,{'spotify': 'https://open.spotify.com/artist/3aPeCAQ5XK9CLy0xmI0jzo'},144,[],https://api.spotify.com/v1/artists/3aPeCAQ5XK9CLy0xmI0jzo,3aPeCAQ5XK9CLy0xmI0jzo,Fam Band,28.0
3,{'spotify': 'https://open.spotify.com/artist/2VfhnjHV7e3M499VPa1w71'},062,[],https://api.spotify.com/v1/artists/2VfhnjHV7e3M499VPa1w71,2VfhnjHV7e3M499VPa1w71,Saving Escape,21.0
4,{'spotify': 'https://open.spotify.com/artist/5IYB6AHoUTP9zukPC2boFS'},904,[roots americana],https://api.spotify.com/v1/artists/5IYB6AHoUTP9zukPC2boFS,5IYB6AHoUTP9zukPC2boFS,The Tillers,19.0


In [114]:
# trim_spotify_clean.dtypes
# trim_spotify_clean["popularity"] = pd.to_numeric(trim_spotify_clean["popularity"])
trim_spotify_clean["popularity"] = trim_spotify_clean["popularity"].apply(pd.to_numeric, errors='coerce')
# trim_spotify_clean['order'] = trim_spotify_clean.Index
trim_spotify_local = trim_spotify_clean
trim_spotify_local.reset_index(inplace=True)
trim_spotify_local['order'] = trim_spotify_local['index']
trim_spotify_local

,level_0,index,external_urls,followers,genres,href,id,name,popularity,order,local
0,0,0,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},520,[],https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,Trauma Illinois,4.0,0,yes
1,1,1,{'spotify': 'https://open.spotify.com/artist/4Fo2qgLwTad6ICYbIu7CQR'},747,[movie tunes],https://api.spotify.com/v1/artists/4Fo2qgLwTad6ICYbIu7CQR,4Fo2qgLwTad6ICYbIu7CQR,Ashley Lilley,57.0,1,yes
2,2,2,{'spotify': 'https://open.spotify.com/artist/3aPeCAQ5XK9CLy0xmI0jzo'},144,[],https://api.spotify.com/v1/artists/3aPeCAQ5XK9CLy0xmI0jzo,3aPeCAQ5XK9CLy0xmI0jzo,Fam Band,28.0,2,yes
3,3,3,{'spotify': 'https://open.spotify.com/artist/2VfhnjHV7e3M499VPa1w71'},062,[],https://api.spotify.com/v1/artists/2VfhnjHV7e3M499VPa1w71,2VfhnjHV7e3M499VPa1w71,Saving Escape,21.0,3,yes
4,4,4,{'spotify': 'https://open.spotify.com/artist/5IYB6AHoUTP9zukPC2boFS'},904,[roots americana],https://api.spotify.com/v1/artists/5IYB6AHoUTP9zukPC2boFS,5IYB6AHoUTP9zukPC2boFS,The Tillers,19.0,4,yes
...,...,...,...,...,...,...,...,...,...,...,...
752,752,752,{'spotify': 'https://open.spotify.com/artist/6TAV25x5AHhy4hbaBzr64c'},95,[],https://api.spotify.com/v1/artists/6TAV25x5AHhy4hbaBzr64c,6TAV25x5AHhy4hbaBzr64c,Ben Knight & the Welldiggers,1.0,752,yes
753,753,753,{'spotify': 'https://open.spotify.com/artist/3xd1QDGm37Pblfqn5qR8P6'},54,[],https://api.spotify.com/v1/artists/3xd1QDGm37Pblfqn5qR8P6,3xd1QDGm37Pblfqn5qR8P6,Warrick & Lowell,0.0,753,yes
754,754,754,{'spotify': 'https://open.spotify.com/artist/42ufGf70mJ6FERFQMBWauz'},: 0,[],https://api.spotify.com/v1/artists/42ufGf70mJ6FERFQMBWauz,42ufGf70mJ6FERFQMBWauz,Nicholas Clark,0.0,754,yes
755,755,755,{'spotify': 'https://open.spotify.com/artist/2lELmBTpxqW7hYmgw64YgE'},61,[],https://api.spotify.com/v1/artists/2lELmBTpxqW7hYmgw64YgE,2lELmBTpxqW7hYmgw64YgE,Chelsea Ford & the Trouble,0.0,755,yes


In [137]:
# import numpy as np
# trim_spotify_local['local'] = np.where((trim_spotify_local['order']>400) & (trim_spotify_local['popularity']>20) , 'no', 'yes')
# trim_to_local_only = trim_spotify_local[trim_spotify_local['local'] =='yes'] 
# trim_to_local_only

In [138]:
# # THIS IS A BAD RULE, GET RID OF THIS
# trim_spotify_local[trim_spotify_local['local'] =='no'] 

In [195]:
trim_to_local_only = trim_spotify_local
# Reassign to correct DF
trim_spotify_clean = trim_to_local_only

trim_spotify_clean = trim_spotify_clean.replace(': ','', regex=True)
trim_spotify_clean[['followers']] = trim_spotify_clean[['followers']].astype(float)

goodbye_national_list = ['Billie Eilish','Marc Anthony','Travis Barker','The National',
'Supertramp','A Day To Remember','Nancy Sinatra','Moira Dela Torre',
'Audrey Mika', 'BANKS','Vacations','Patsy Cline','Ricky Nelson',
'SOYEON of (G)I-DLE','Jameson Rodgers','RetroVision','Zooey Deschanel',
'Eurielle',	'Ashley Lilley','Rucci','Aaryan Shah','Hilary Hahn',
'The Original Broadway Cast Of School Of Rock','Alter Ego','Madleen Kane',
'Loren Hartman','Rarin','Ted Nugent','Wilderado','Buckethead','Ghost Rider',
'Celestial Aeon Project','Copper King','JUDGE','Greer','The Sweeplings','Focus','McEnroe','	Matt Fuze',
'Voodoo Glow Skulls','Louis Langrée','Here Come The Mummies','The Shivers','Sungazer','Ben Flocks','The Shivers','Night Owl',
'Blueprint','Drew Schueler','Kelly Thomas','Sundae Drives','Meridian','Dark Matters','Mitch De Klein','Mick Blankenship',
'Wayward Sons','Dan Campbell','Soften the Glare','Bad Veins','Continuados','','','',
'','','','','','','','',
'','','','','','','',''
]

pd.set_option('display.max_rows', 500)

trim_spotify_clean_er = trim_spotify_clean[~trim_spotify_clean['name'].isin(goodbye_national_list)]
trim_spotify_clean_er = trim_spotify_clean_er.drop_duplicates('name')
# trim_spotify_clean_er.sort_values('followers',ascending=False, na_position='last').head(40)
trim_spotify_clean_er.sort_values('popularity',ascending=False, na_position='last').head(40)

,level_0,index,external_urls,followers,genres,href,id,name,popularity,order,local
5,5,5,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},488.0,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,5,yes
686,686,686,{'spotify': 'https://open.spotify.com/artist/7GN9PivdemQRKjDt4z5Zv8'},536.0,"[alternative r&b, indie soul, lgbtq+ hip hop, neo soul]",https://api.spotify.com/v1/artists/7GN9PivdemQRKjDt4z5Zv8,7GN9PivdemQRKjDt4z5Zv8,The Internet,65.0,686,no
737,737,737,{'spotify': 'https://open.spotify.com/artist/58bPSgeKpyyFlJ9LatULIO'},235.0,"[cincinnati indie, modern rock]",https://api.spotify.com/v1/artists/58bPSgeKpyyFlJ9LatULIO,58bPSgeKpyyFlJ9LatULIO,HARBOUR,51.0,737,no
58,58,58,{'spotify': 'https://open.spotify.com/artist/0o9qzOHuHkGZ3xMFKpMFkC'},242.0,"[brostep, canadian electronic, dark clubbing, dubstep, edm, electro house, electro trash, electronic trap]",https://api.spotify.com/v1/artists/0o9qzOHuHkGZ3xMFKpMFkC,0o9qzOHuHkGZ3xMFKpMFkC,Black Tiger Sex Machine,51.0,58,yes
9,9,9,{'spotify': 'https://open.spotify.com/artist/4bnCmwKU89Rt1zWnRIdSUa'},609.0,"[alternative country, deep new americana, new americana, roots americana, southern americana]",https://api.spotify.com/v1/artists/4bnCmwKU89Rt1zWnRIdSUa,4bnCmwKU89Rt1zWnRIdSUa,Arlo McKinley,45.0,9,yes
6,6,6,{'spotify': 'https://open.spotify.com/artist/5qRXoCC1CDTMTli9gplUvk'},652.0,"[cincinnati indie, piano rock]",https://api.spotify.com/v1/artists/5qRXoCC1CDTMTli9gplUvk,5qRXoCC1CDTMTli9gplUvk,Foxy Shazam,43.0,6,yes
436,436,436,{'spotify': 'https://open.spotify.com/artist/0XL0fuHaYwcL0JGouIxXid'},228.0,[],https://api.spotify.com/v1/artists/0XL0fuHaYwcL0JGouIxXid,0XL0fuHaYwcL0JGouIxXid,Matt Fuze,43.0,436,no
76,76,76,{'spotify': 'https://open.spotify.com/artist/70fUpxdAr6t0LJw3xJmMhm'},977.0,"[cincinnati indie, indiecoustica, stomp and holler]",https://api.spotify.com/v1/artists/70fUpxdAr6t0LJw3xJmMhm,70fUpxdAr6t0LJw3xJmMhm,Motherfolk,41.0,76,yes
94,94,94,{'spotify': 'https://open.spotify.com/artist/6gwexMlbo78wmlBF8U2V6r'},692.0,[indie rockism],https://api.spotify.com/v1/artists/6gwexMlbo78wmlBF8U2V6r,6gwexMlbo78wmlBF8U2V6r,Jane Decker,41.0,94,yes
42,42,42,{'spotify': 'https://open.spotify.com/artist/1z26dqCz3PySR4G7HWsn93'},597.0,"[ectofolk, folk]",https://api.spotify.com/v1/artists/1z26dqCz3PySR4G7HWsn93,1z26dqCz3PySR4G7HWsn93,Over The Rhine,41.0,42,yes


In [196]:
ids_only = trim_spotify_clean_er['id']
ids_only = [x for x in ids_only if str(x) != 'nan']
ids_only = list(set(ids_only))
ids_only

with open('list_ids.txt', 'w') as f:
    for item in ids_only:
        f.write("%s\n" % item)

In [271]:
df_bands.head()

,names
0,Trauma Illinois
1,Ashley Lilly
2,Fam Band
3,Saving Escape
4,The Tillers


In [272]:
df_cincy_spotify = pd.merge(df_bands,trim_spotify_clean_er, left_on='names',right_on='name')
df_cincy_spotify.head()

,names,level_0,index,external_urls,followers,genres,href,id,name,popularity,order,local
0,Trauma Illinois,0,0,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},520.0,[],https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,Trauma Illinois,4.0,0,yes
1,Fam Band,2,2,{'spotify': 'https://open.spotify.com/artist/3aPeCAQ5XK9CLy0xmI0jzo'},144.0,[],https://api.spotify.com/v1/artists/3aPeCAQ5XK9CLy0xmI0jzo,3aPeCAQ5XK9CLy0xmI0jzo,Fam Band,28.0,2,yes
2,Saving Escape,3,3,{'spotify': 'https://open.spotify.com/artist/2VfhnjHV7e3M499VPa1w71'},62.0,[],https://api.spotify.com/v1/artists/2VfhnjHV7e3M499VPa1w71,2VfhnjHV7e3M499VPa1w71,Saving Escape,21.0,3,yes
3,The Tillers,4,4,{'spotify': 'https://open.spotify.com/artist/5IYB6AHoUTP9zukPC2boFS'},904.0,[roots americana],https://api.spotify.com/v1/artists/5IYB6AHoUTP9zukPC2boFS,5IYB6AHoUTP9zukPC2boFS,The Tillers,19.0,4,yes
4,PUBLIC,5,5,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},488.0,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,5,yes


In [273]:
df_bands.describe()

,names
count,483
unique,481
top,Hillary Hahn
freq,2


In [274]:
df_cincy_spotify.dtypes

names            object 
level_0          int64  
index            int64  
external_urls    object 
followers        float64
genres           object 
href             object 
id               object 
name             object 
popularity       float64
order            int64  
local            object 
dtype: object

In [275]:
df_cincy_spotify['genres'] = df_cincy_spotify['genres'][1:]
df_cincy_spotify['genres'] = df_cincy_spotify['genres'][:-1]

## Clean Up Column Data


In [276]:
# df_cincy_spotify[['GENRE1','GENRE2','GENRE3']] = df_cincy_spotify['genres'].str.split(',', n=2, expand=True)
# df_cincy_spotify.head()

In [277]:
# df_cincy_spotify_genres

In [278]:
# #if one column DataFrame
# df_cincy_spotify_genres = pd.DataFrame(df_cincy_spotify['genres'].values.tolist(), index=df_cincy_spotify.index)

# #if multiple columns and need join new df to existing
# df_cincy_spotify = df_cincy_spotify.join(pd.DataFrame(df_cincy_spotify_genres.pop('genres').values.tolist(), index=df_cincy_spotify.index))

In [279]:
# df_cincy_spotify['GENRE1'] = df_cincy_spotify['genres'].str[0]
# df_cincy_spotify['GENRE2'] = df_cincy_spotify['genres'].str[0]
# df_cincy_spotify['GENRE3'] = df_cincy_spotify['genres'].str[0]

In [291]:
# df_cincy_spotify[['GENRE1','GENRE2','GENRE3']] = df_cincy_spotify['genres'].str.split(",",expand=True)
for i in range(len(df_cincy_spotify)):
    try:
        df_cincy_spotify['GENRE1'] = df_cincy_spotify['genres'].str[0]
    except:
        'NA'
for i in range(len(df_cincy_spotify)):
    try:
        df_cincy_spotify['GENRE2'] = df_cincy_spotify['genres'].str[1]
    except:
        'NA'
for i in range(len(df_cincy_spotify)):
    try:
        df_cincy_spotify['GENRE3'] = df_cincy_spotify['genres'].str[2]
    except:
        'NA'

df_cincy_spotify.head(5)

,names,level_0,index,external_urls,followers,genres,href,id,name,popularity,order,local,GENRE1,GENRE2,GENRE3
0,Trauma Illinois,0,0,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},520.0,NaN,https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,Trauma Illinois,4.0,0,yes,NaN,NaN,NaN
1,Fam Band,2,2,{'spotify': 'https://open.spotify.com/artist/3aPeCAQ5XK9CLy0xmI0jzo'},144.0,[],https://api.spotify.com/v1/artists/3aPeCAQ5XK9CLy0xmI0jzo,3aPeCAQ5XK9CLy0xmI0jzo,Fam Band,28.0,2,yes,NaN,NaN,NaN
2,Saving Escape,3,3,{'spotify': 'https://open.spotify.com/artist/2VfhnjHV7e3M499VPa1w71'},62.0,[],https://api.spotify.com/v1/artists/2VfhnjHV7e3M499VPa1w71,2VfhnjHV7e3M499VPa1w71,Saving Escape,21.0,3,yes,NaN,NaN,NaN
3,The Tillers,4,4,{'spotify': 'https://open.spotify.com/artist/5IYB6AHoUTP9zukPC2boFS'},904.0,[roots americana],https://api.spotify.com/v1/artists/5IYB6AHoUTP9zukPC2boFS,5IYB6AHoUTP9zukPC2boFS,The Tillers,19.0,4,yes,roots americana,NaN,NaN
4,PUBLIC,5,5,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},488.0,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,5,yes,indie rockism,NaN,NaN


In [283]:
df_cincy_spotify_clean = df_cincy_spotify
df_cincy_spotify_clean.dtypes

names            object 
level_0          int64  
index            int64  
external_urls    object 
followers        float64
genres           object 
href             object 
id               object 
name             object 
popularity       float64
order            int64  
local            object 
GENRE1           object 
GENRE2           object 
GENRE3           object 
dtype: object

In [284]:
df_cincy_spotify_clean['followers'] = df_cincy_spotify_clean['followers'].apply(pd.to_numeric, errors='coerce')
df_cincy_spotify_clean['popularity'] = df_cincy_spotify_clean['popularity'].apply(pd.to_numeric, errors='coerce')
df_cincy_spotify_clean.dtypes

names            object 
level_0          int64  
index            int64  
external_urls    object 
followers        float64
genres           object 
href             object 
id               object 
name             object 
popularity       float64
order            int64  
local            object 
GENRE1           object 
GENRE2           object 
GENRE3           object 
dtype: object

In [285]:
df_cincy_spotify_clean.sort_values('followers', ascending=False).head(10)

,names,level_0,index,external_urls,followers,genres,href,id,name,popularity,order,local,GENRE1,GENRE2,GENRE3
93,Carriers,135,135,{'spotify': 'https://open.spotify.com/artist/03Y6dXzmx4DFtBblTZpAw4'},981.0,[],https://api.spotify.com/v1/artists/03Y6dXzmx4DFtBblTZpAw4,03Y6dXzmx4DFtBblTZpAw4,Carriers,13.0,135,yes,NaN,NaN,NaN
53,Motherfolk,76,76,{'spotify': 'https://open.spotify.com/artist/70fUpxdAr6t0LJw3xJmMhm'},977.0,"[cincinnati indie, indiecoustica, stomp and holler]",https://api.spotify.com/v1/artists/70fUpxdAr6t0LJw3xJmMhm,70fUpxdAr6t0LJw3xJmMhm,Motherfolk,41.0,76,yes,cincinnati indie,indiecoustica,stomp and holler
45,Lauren Eylise,62,62,{'spotify': 'https://open.spotify.com/artist/12dqbtoVLwnXTJfK2XpaJb'},962.0,[],https://api.spotify.com/v1/artists/12dqbtoVLwnXTJfK2XpaJb,12dqbtoVLwnXTJfK2XpaJb,Lauren Eylise,18.0,62,yes,NaN,NaN,NaN
135,Valdrin,208,208,{'spotify': 'https://open.spotify.com/artist/4KsrkYZYtvXJJeSZgE5jBU'},953.0,"[melodic black metal, voidgaze]",https://api.spotify.com/v1/artists/4KsrkYZYtvXJJeSZgE5jBU,4KsrkYZYtvXJJeSZgE5jBU,Valdrin,15.0,208,yes,melodic black metal,voidgaze,NaN
27,In The Pines,39,39,{'spotify': 'https://open.spotify.com/artist/3KQoZQIHTfe9FsMMRuw9S6'},951.0,[cincinnati indie],https://api.spotify.com/v1/artists/3KQoZQIHTfe9FsMMRuw9S6,3KQoZQIHTfe9FsMMRuw9S6,In The Pines,22.0,39,yes,cincinnati indie,NaN,NaN
28,Cincinnati Pops Orchestra,41,41,{'spotify': 'https://open.spotify.com/artist/1FV3sKugQJFGWgWpMIA2Fy'},929.0,"[american orchestra, pops orchestra]",https://api.spotify.com/v1/artists/1FV3sKugQJFGWgWpMIA2Fy,1FV3sKugQJFGWgWpMIA2Fy,Cincinnati Pops Orchestra,35.0,41,yes,american orchestra,pops orchestra,NaN
3,The Tillers,4,4,{'spotify': 'https://open.spotify.com/artist/5IYB6AHoUTP9zukPC2boFS'},904.0,[roots americana],https://api.spotify.com/v1/artists/5IYB6AHoUTP9zukPC2boFS,5IYB6AHoUTP9zukPC2boFS,The Tillers,19.0,4,yes,roots americana,NaN,NaN
233,The String Theory,427,427,{'spotify': 'https://open.spotify.com/artist/6gT9lfBqZ1R9BrSEuB09E1'},859.0,[],https://api.spotify.com/v1/artists/6gT9lfBqZ1R9BrSEuB09E1,6gT9lfBqZ1R9BrSEuB09E1,The String Theory,34.0,427,no,NaN,NaN,NaN
42,Sylmar,57,57,{'spotify': 'https://open.spotify.com/artist/6ZyaO6lPHxQMapy52NBzHp'},803.0,[cincinnati indie],https://api.spotify.com/v1/artists/6ZyaO6lPHxQMapy52NBzHp,6ZyaO6lPHxQMapy52NBzHp,Sylmar,22.0,57,yes,cincinnati indie,NaN,NaN
58,Ghost Train,83,83,{'spotify': 'https://open.spotify.com/artist/7o9mgsmNx97zJa41WBDNqx'},788.0,[],https://api.spotify.com/v1/artists/7o9mgsmNx97zJa41WBDNqx,7o9mgsmNx97zJa41WBDNqx,Ghost Train,18.0,83,yes,NaN,NaN,NaN


In [286]:
df_cincy_spotify_clean.sort_values('popularity', ascending=False).head(10)

,names,level_0,index,external_urls,followers,genres,href,id,name,popularity,order,local,GENRE1,GENRE2,GENRE3
4,PUBLIC,5,5,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},488.0,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,5,yes,indie rockism,NaN,NaN
8,Arlo McKinley,9,9,{'spotify': 'https://open.spotify.com/artist/4bnCmwKU89Rt1zWnRIdSUa'},609.0,"[alternative country, deep new americana, new americana, roots americana, southern americana]",https://api.spotify.com/v1/artists/4bnCmwKU89Rt1zWnRIdSUa,4bnCmwKU89Rt1zWnRIdSUa,Arlo McKinley,45.0,9,yes,alternative country,deep new americana,new americana
5,Foxy Shazam,6,6,{'spotify': 'https://open.spotify.com/artist/5qRXoCC1CDTMTli9gplUvk'},652.0,"[cincinnati indie, piano rock]",https://api.spotify.com/v1/artists/5qRXoCC1CDTMTli9gplUvk,5qRXoCC1CDTMTli9gplUvk,Foxy Shazam,43.0,6,yes,cincinnati indie,piano rock,NaN
29,Over The Rhine,42,42,{'spotify': 'https://open.spotify.com/artist/1z26dqCz3PySR4G7HWsn93'},597.0,"[ectofolk, folk]",https://api.spotify.com/v1/artists/1z26dqCz3PySR4G7HWsn93,1z26dqCz3PySR4G7HWsn93,Over The Rhine,41.0,42,yes,ectofolk,folk,NaN
53,Motherfolk,76,76,{'spotify': 'https://open.spotify.com/artist/70fUpxdAr6t0LJw3xJmMhm'},977.0,"[cincinnati indie, indiecoustica, stomp and holler]",https://api.spotify.com/v1/artists/70fUpxdAr6t0LJw3xJmMhm,70fUpxdAr6t0LJw3xJmMhm,Motherfolk,41.0,76,yes,cincinnati indie,indiecoustica,stomp and holler
65,Jane Decker,94,94,{'spotify': 'https://open.spotify.com/artist/6gwexMlbo78wmlBF8U2V6r'},692.0,[indie rockism],https://api.spotify.com/v1/artists/6gwexMlbo78wmlBF8U2V6r,6gwexMlbo78wmlBF8U2V6r,Jane Decker,41.0,94,yes,indie rockism,NaN,NaN
36,ENMY,49,49,{'spotify': 'https://open.spotify.com/artist/0DJJrg8eUagEWZXFgwKJfM'},457.0,[],https://api.spotify.com/v1/artists/0DJJrg8eUagEWZXFgwKJfM,0DJJrg8eUagEWZXFgwKJfM,ENMY,40.0,49,yes,NaN,NaN,NaN
91,Calumet,133,133,{'spotify': 'https://open.spotify.com/artist/0XIAEZvFmaxsJwhs5CvSAU'},136.0,[],https://api.spotify.com/v1/artists/0XIAEZvFmaxsJwhs5CvSAU,0XIAEZvFmaxsJwhs5CvSAU,Calumet,36.0,133,yes,NaN,NaN,NaN
28,Cincinnati Pops Orchestra,41,41,{'spotify': 'https://open.spotify.com/artist/1FV3sKugQJFGWgWpMIA2Fy'},929.0,"[american orchestra, pops orchestra]",https://api.spotify.com/v1/artists/1FV3sKugQJFGWgWpMIA2Fy,1FV3sKugQJFGWgWpMIA2Fy,Cincinnati Pops Orchestra,35.0,41,yes,american orchestra,pops orchestra,NaN
233,The String Theory,427,427,{'spotify': 'https://open.spotify.com/artist/6gT9lfBqZ1R9BrSEuB09E1'},859.0,[],https://api.spotify.com/v1/artists/6gT9lfBqZ1R9BrSEuB09E1,6gT9lfBqZ1R9BrSEuB09E1,The String Theory,34.0,427,no,NaN,NaN,NaN


In [287]:
len(df_cincy_spotify_clean)

428

In [288]:
df_cincy_spotify_clean_no_dups = df_cincy_spotify_clean.loc[df_cincy_spotify_clean.astype(str).drop_duplicates('name').index]
len(df_cincy_spotify_clean_no_dups)

253

### Write Out Artist Info

In [289]:
df_cincy_spotify_clean_no_dups.to_csv("artists_info.csv", sep = ',')

In [290]:
df_cincy_spotify_clean_no_dups.sort_values('popularity',ascending=False, na_position='last')

,names,level_0,index,external_urls,followers,genres,href,id,name,popularity,order,local,GENRE1,GENRE2,GENRE3
4,PUBLIC,5,5,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},488.0,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,5,yes,indie rockism,NaN,NaN
8,Arlo McKinley,9,9,{'spotify': 'https://open.spotify.com/artist/4bnCmwKU89Rt1zWnRIdSUa'},609.0,"[alternative country, deep new americana, new americana, roots americana, southern americana]",https://api.spotify.com/v1/artists/4bnCmwKU89Rt1zWnRIdSUa,4bnCmwKU89Rt1zWnRIdSUa,Arlo McKinley,45.0,9,yes,alternative country,deep new americana,new americana
5,Foxy Shazam,6,6,{'spotify': 'https://open.spotify.com/artist/5qRXoCC1CDTMTli9gplUvk'},652.0,"[cincinnati indie, piano rock]",https://api.spotify.com/v1/artists/5qRXoCC1CDTMTli9gplUvk,5qRXoCC1CDTMTli9gplUvk,Foxy Shazam,43.0,6,yes,cincinnati indie,piano rock,NaN
65,Jane Decker,94,94,{'spotify': 'https://open.spotify.com/artist/6gwexMlbo78wmlBF8U2V6r'},692.0,[indie rockism],https://api.spotify.com/v1/artists/6gwexMlbo78wmlBF8U2V6r,6gwexMlbo78wmlBF8U2V6r,Jane Decker,41.0,94,yes,indie rockism,NaN,NaN
53,Motherfolk,76,76,{'spotify': 'https://open.spotify.com/artist/70fUpxdAr6t0LJw3xJmMhm'},977.0,"[cincinnati indie, indiecoustica, stomp and holler]",https://api.spotify.com/v1/artists/70fUpxdAr6t0LJw3xJmMhm,70fUpxdAr6t0LJw3xJmMhm,Motherfolk,41.0,76,yes,cincinnati indie,indiecoustica,stomp and holler
29,Over The Rhine,42,42,{'spotify': 'https://open.spotify.com/artist/1z26dqCz3PySR4G7HWsn93'},597.0,"[ectofolk, folk]",https://api.spotify.com/v1/artists/1z26dqCz3PySR4G7HWsn93,1z26dqCz3PySR4G7HWsn93,Over The Rhine,41.0,42,yes,ectofolk,folk,NaN
36,ENMY,49,49,{'spotify': 'https://open.spotify.com/artist/0DJJrg8eUagEWZXFgwKJfM'},457.0,[],https://api.spotify.com/v1/artists/0DJJrg8eUagEWZXFgwKJfM,0DJJrg8eUagEWZXFgwKJfM,ENMY,40.0,49,yes,NaN,NaN,NaN
91,Calumet,133,133,{'spotify': 'https://open.spotify.com/artist/0XIAEZvFmaxsJwhs5CvSAU'},136.0,[],https://api.spotify.com/v1/artists/0XIAEZvFmaxsJwhs5CvSAU,0XIAEZvFmaxsJwhs5CvSAU,Calumet,36.0,133,yes,NaN,NaN,NaN
28,Cincinnati Pops Orchestra,41,41,{'spotify': 'https://open.spotify.com/artist/1FV3sKugQJFGWgWpMIA2Fy'},929.0,"[american orchestra, pops orchestra]",https://api.spotify.com/v1/artists/1FV3sKugQJFGWgWpMIA2Fy,1FV3sKugQJFGWgWpMIA2Fy,Cincinnati Pops Orchestra,35.0,41,yes,american orchestra,pops orchestra,NaN
233,The String Theory,427,427,{'spotify': 'https://open.spotify.com/artist/6gT9lfBqZ1R9BrSEuB09E1'},859.0,[],https://api.spotify.com/v1/artists/6gT9lfBqZ1R9BrSEuB09E1,6gT9lfBqZ1R9BrSEuB09E1,The String Theory,34.0,427,no,NaN,NaN,NaN
